In [1]:
# Change directory to the root of the project
import os 
os.chdir('..')

import wandb
import torch
from applications.tic_tac_toe.train import train

In [2]:
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
    },
    'parameters': {
        # Optimizer parameters
        'learning_rate': {
            'distribution': 'log_uniform_values',
            'min': 0.00001,
            'max': 0.1
        },
        'weight_decay': {
            'distribution': 'log_uniform_values',
            'min': 0.00001,
            'max': 0.1
        },

        # Model parameters
        'attention_layers': {
            'values': [1, 2, 3, 4]
        },
        'transformer_size': {
            'values': ['tiny', 'small', 'medium', 'large', 'xlarge']
        },
        'dropout': {
            'values': [0.0, 0.1, 0.01, 0.001, 0.0001]
        },
        'norm_first': {
            'values': [True, False]
        },
        'activation': {
            'values': ['relu', 'gelu']
        },

        # Trainer parameters
        'replay_buffer_max_size': {
            'value': 10000
        },
        'value_softness': {
            'distribution': 'uniform',
            'min': 0.0,
            'max': 1.0
        },
        'mask_illegal_moves': {
            'values': [True, False]
        },
        'mask_value': {
            'values': [-20.0, -15.0, -10.0, -5.0]
        },

        # Training parameters
        'num_iterations': {
            'value': 100
        },
        'games_per_iteration': {
            'value': 10
        },
        'batch_size': {
            'values': [128, 256, 512, 1024]
        },
        'steps_per_iteration': {
            'value': 100
        },
        'num_simulations': {
            'values': [100]
        },
        'checkpoint_frequency': {
            'value': 20
        }
    }
}

# Transformer size mapping
transformer_size_mapping = {
    'tiny': { 'embed_dim': 4, 'num_heads': 1, 'feedforward_dim': 16 },
    'small': { 'embed_dim': 8, 'num_heads': 2, 'feedforward_dim': 32 },
    'medium': { 'embed_dim': 16, 'num_heads': 4, 'feedforward_dim': 64 },
    'large': { 'embed_dim': 32, 'num_heads': 8, 'feedforward_dim': 128 },
    'xlarge': { 'embed_dim': 64, 'num_heads': 16, 'feedforward_dim': 256 }
}


In [3]:
# Some default parameters

from core.implementations.AlphaZero import AlphaZeroConfig

# AlphaZero parameters
alphazero_config = AlphaZeroConfig(
    exploration_constant=1.0,
    dirichlet_alpha=0.3,
    dirichlet_epsilon=0.25,
    temperature=1.0
)

# AlphaZero evaluation parameters
alphazero_eval_config = AlphaZeroConfig(
    exploration_constant=1.0,
    dirichlet_alpha=0.0,
    dirichlet_epsilon=0.0,
    temperature=0.0
)

In [4]:
from applications.tic_tac_toe.transformer_model import TicTacToeTransformerInterface

def sweep_agent():
    with wandb.init(project='AlphaZero-TicTacToe') as run:
        config = {
            'model_type': 'transformer',
            'model_params': {
                'attention_layers': run.config.attention_layers,
                **transformer_size_mapping[run.config.transformer_size]
            },
            'device': 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu',
            'tree_search_params': alphazero_config,
            'tree_search_eval_params': alphazero_eval_config,
            'trainer_params': {
                'replay_buffer_max_size': run.config.replay_buffer_max_size,
                'value_softness': run.config.value_softness,
                'mask_illegal_moves': run.config.mask_illegal_moves,
                'mask_value': run.config.mask_value
            },
            'optimizer_params': {
                'lr': run.config.learning_rate,
                'betas': (0.9, 0.999),
                'eps': 1e-8,
                'weight_decay': run.config.weight_decay,
                'amsgrad': False
            },
            'training_params': {
                'num_iterations': run.config.num_iterations,
                'games_per_iteration': run.config.games_per_iteration,
                'batch_size': run.config.batch_size,
                'steps_per_iteration': run.config.steps_per_iteration,
                'num_simulations': run.config.num_simulations,
                'checkpoint_frequency': run.config.checkpoint_frequency
            }
        }

        model = TicTacToeTransformerInterface(
            device=config['device'],
            **config['model_params']
        )

        # Use training script
        train(
            config=config,
            model=model,
            use_wandb=True,
            wandb_watch_params={
                'watch': True,
                'log': 'all',
                'log_freq': 100,
                'log_graph': True
            },
            wandb_run=run
        )

In [5]:
sweep_id = wandb.sweep(
    sweep=sweep_config,
    project='AlphaZero-TicTacToe',
    entity='eigenway',
)


wandb.agent(
    sweep_id,
    function=sweep_agent,
    count=50
)

Create sweep with ID: z91b8lti
Sweep URL: https://wandb.ai/eigenway/AlphaZero-TicTacToe/sweeps/z91b8lti


wandb: Agent Starting Run: 1tvknncm with config:
wandb: 	activation: relu
wandb: 	attention_layers: 2
wandb: 	batch_size: 512
wandb: 	checkpoint_frequency: 20
wandb: 	dropout: 0.01
wandb: 	games_per_iteration: 10
wandb: 	learning_rate: 0.0007647649282292424
wandb: 	mask_illegal_moves: False
wandb: 	mask_value: -10
wandb: 	norm_first: True
wandb: 	num_iterations: 100
wandb: 	num_simulations: 100
wandb: 	replay_buffer_max_size: 10000
wandb: 	steps_per_iteration: 100
wandb: 	transformer_size: xlarge
wandb: 	value_softness: 0.27162888872040936
wandb: 	weight_decay: 0.005288860670490674
wandb: Currently logged in as: eohjelle (eigenway) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Training model: transformer
Using device: mps

Iteration 1/100
Self-play phase...
Playing game 10/10
Generated 98 new positions
Training phase...

Iteration 1 summary:
Average loss: 2.6434
Average policy_loss: 1.8133
Average value_loss: 0.8301
Replay buffer size: 98
Time taken: 6.7s

Iteration 2/100
Self-play phase...
Playing game 10/10
Generated 90 new positions
Training phase...

Iteration 2 summary:
Average loss: 1.8988
Average policy_loss: 1.1526
Average value_loss: 0.7461
Replay buffer size: 188
Time taken: 9.1s

Iteration 3/100
Self-play phase...
Playing game 10/10
Generated 94 new positions
Training phase...

Iteration 3 summary:
Average loss: 1.4862
Average policy_loss: 0.8457
Average value_loss: 0.6405
Replay buffer size: 282
Time taken: 8.4s

Iteration 4/100
Self-play phase...
Playing game 10/10
Generated 94 new positions
Training phase...

Iteration 4 summary:
Average loss: 1.4541
Average policy_loss: 0.9220
Average value_loss: 0.5321
Replay buffer size: 376
Time taken: 12.2

buffer_size,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
iteration_time,▁▁▂▆▂▆▂▂▂▂▂▂▂▇▂▂▇▂▂▃▇▂▃▃▂▂▃█▃▃▃█▃▃▃▃▃▃▃▃
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
num_games,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
num_positions,▆▇▆█▅▄▄▄▂▃▂▁▄▃▅▃▂▃▄▂▄▅▄▂▄▆▄▅▆▆▅▅█▄▄▅▄▅▄▄
policy_loss,█▃▂▁▁▁▁▂▂▂▂▂▃▃▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
value_loss,█▅▃▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_win_rate,1.3
buffer_size,8519
iteration_time,50.30384
loss,1.18897


...

Training model: transformer
Using device: mps

Iteration 1/100
Self-play phase...
Playing game 10/10
Generated 90 new positions
Training phase...

Iteration 1 summary:
Average loss: 1.8282
Average policy_loss: 1.3740
Average value_loss: 0.4541
Replay buffer size: 90
Time taken: 9.4s

Iteration 2/100
Self-play phase...
Playing game 10/10
Generated 92 new positions
Training phase...

Iteration 2 summary:
Average loss: 0.9463
Average policy_loss: 0.7601
Average value_loss: 0.1862
Replay buffer size: 182
Time taken: 13.7s

Iteration 3/100
Self-play phase...
Playing game 10/10
Generated 81 new positions
Training phase...

Iteration 3 summary:
Average loss: 0.9888
Average policy_loss: 0.8126
Average value_loss: 0.1762
Replay buffer size: 263
Time taken: 17.5s

Iteration 4/100
Self-play phase...
Playing game 10/10
Generated 88 new positions
Training phase...

Iteration 4 summary:
Average loss: 1.0515
Average policy_loss: 0.8967
Average value_loss: 0.1548
Replay buffer size: 351
Time taken: 20

buffer_size,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇██
iteration_time,▁▂▇▂▂▆▂▂▂▂▂▇▂▂▂▂▂▂▂▂▇▂▂▂▂▇▂▂█▃▃▃▃▂█▃▂█▃▃
loss,█▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
num_games,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
num_positions,▆▇▃▅▇▂▄▂▃▁▂▃▄▄▄▅▅█▄▅▅▅▇▅▇██▇▇▂▇▅▇▅▇▅▅▆▅▆
policy_loss,▁▆▅▆██▇▇▇▇▆▆▆▆▆▅▆▆▅▅▅▅▅▅▅▅▅▅▅▆▅▆▅▅▅▅▅▅▅▅
value_loss,█▂▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_win_rate,1.25
buffer_size,8658
iteration_time,68.67727
loss,1.04205


wandb: Agent Starting Run: 7c3e98y3 with config:
wandb: 	activation: gelu
wandb: 	attention_layers: 4
wandb: 	batch_size: 512
wandb: 	checkpoint_frequency: 20
wandb: 	dropout: 0.0001
wandb: 	games_per_iteration: 10
wandb: 	learning_rate: 0.0443884942945996
wandb: 	mask_illegal_moves: True
wandb: 	mask_value: -5
wandb: 	norm_first: True
wandb: 	num_iterations: 100
wandb: 	num_simulations: 100
wandb: 	replay_buffer_max_size: 10000
wandb: 	steps_per_iteration: 100
wandb: 	transformer_size: tiny
wandb: 	value_softness: 0.8855538374778479
wandb: 	weight_decay: 0.00011723740862830225


Training model: transformer
Using device: mps

Iteration 1/100
Self-play phase...
Playing game 10/10
Generated 80 new positions
Training phase...

Iteration 1 summary:
Average loss: 1.2178
Average policy_loss: 1.0263
Average value_loss: 0.1915
Replay buffer size: 80
Time taken: 14.6s

Iteration 2/100
Self-play phase...
Playing game 10/10
Generated 87 new positions
Training phase...

Iteration 2 summary:
Average loss: 1.1077
Average policy_loss: 0.9700
Average value_loss: 0.1377
Replay buffer size: 167
Time taken: 15.0s

Iteration 3/100
Self-play phase...
Playing game 10/10
Generated 91 new positions
Training phase...

Iteration 3 summary:
Average loss: 1.0684
Average policy_loss: 0.9676
Average value_loss: 0.1008
Replay buffer size: 258
Time taken: 20.1s

Iteration 4/100
Self-play phase...
Playing game 10/10
Generated 84 new positions
Training phase...

Iteration 4 summary:
Average loss: 1.0728
Average policy_loss: 0.9732
Average value_loss: 0.0996
Replay buffer size: 342
Time taken: 2

buffer_size,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇███
iteration_time,▁▂▂▇▂▂▇▂▂▇▁▁▂▁▇▂▂▁▂▂▂▂▂▂█▂█▂▂█▂▂▂▂▂▂█▂▂█
loss,▇▆▆████▇▇▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▂▁▁▁▁▁
num_games,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
num_positions,▂▅▃▂▂▄▅▅▅▆▆▅▁▅▅▆▇██▅▃▅█▁▃▆▃▇▅▇▅▅▇▇█▄▇▇▇▃
policy_loss,████▇▇▇▆▅▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁
value_loss,█▃▇▆▆▆▅▅▄▄▄▃▃▃▃▃▂▂▂▃▂▂▁▁▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
best_win_rate,1.15
buffer_size,9052
iteration_time,72.10831
loss,0.88509
